### This is the file scraping movie reviews from IMDB, and create 'my_review.csv', which includes movie names and reviews of them

In [350]:
# Here I use BeautifulSoup to scrape movie reviews from IMDB
import urllib.request
from bs4 import BeautifulSoup
import lxml
import os 
import pandas as pd
import numpy as np
import re
os.chdir('/Users/Bowen/Documents/2017Spring/ADS/Spr2017-proj5-grp11-master/data/')

In [351]:
df1=pd.read_csv('data_clean.csv')

In [352]:
link=[a.replace('?ref_=fn_tt_tt_1','reviews?ref_=tt_urv') for a in df1['movie_imdb_link']]

In [353]:
dic={a.replace('\xa0',''):b for (a,b) in list(zip(df1['movie_title'],link))}
#df2=df1.drop_duplicates(['movie_title'], keep='first')
#df2.to_csv('data_clean.csv')

In [355]:
def getreview(url):
    nf2 = urllib.request.urlopen(url)
    sp = BeautifulSoup(nf2, 'lxml', from_encoding='utf-8')
    b1=sp.findAll('p')
    review=[a.contents for a in b1 if str(a).startswith('<p>')]
    review1=[a for a in review if str(a[0]).startswith('\n')]
    review2=[]
    for a in review1:
        review2=review2+[b for b in a if str(b)!='<br/>']
    review3=[a.replace('\n',' ') for a in review2]
    review4 = [re.sub("[^\w]", " ",  a) for a in review3]
    review5 = [re.sub("\d", "", a).lower() for a in review4]
    review6 = [' '.join(a.split()) for a in review5 if len(a)>6]
    return(review6)

In [ ]:
spider_review=pd.DataFrame(columns=['review_detail','movie_name'])
for i in range(len(dic)):
    key=list(dic.keys())[i]
    url=dic[key]
    reviews=getreview(url)
    chunk=pd.DataFrame({'review_detail':reviews,'movie_name':[key]*len(reviews)})
    spider_review=spider_review.append(chunk)

In [370]:
spider_review=spider_review[['review_detail','movie_name']]
spider_review['movie_name']=[''.join(a.split()) for a in spider_review['movie_name']]
spider_review.to_csv('all_review',index=False)

In [373]:
spider_review.to_csv('my_review.csv',index=False)